# Paper results


In [ ]:
from advection_diffusion_inference_utils import (read_data_files,
                                                 read_experiment_data,
                                                 all_animals,
                                                 all_ears,
                                                 plot_time_series,
                                                 create_time_steps,
                                                 build_grids,
                                                 create_domain_geometry,
                                                 create_PDE_form)


from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
import scipy.stats as sps
import cuqi
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def create_A(data_diff):
    # STEP 2: Read time and location arrays
    #----------------------------------------
    args = data_diff['experiment_par']
    (real_times, real_locations, real_data, real_std_data,
     diff_locations, real_data_diff, real_std_data_diff) = read_data_files(args)
    

    # The left boundary condition is given by the data  
    real_bc_l = real_data.reshape([len(real_locations), len(real_times)])[0,:]

    real_bc_l[real_bc_l<0] = 0

    # The right boundary condition is given by the data (if rbc is not "zero")
    if args.rbc == 'fromData':
        raise Exception('Right boundary condition from data not supported')
    elif args.rbc == 'fromDataClip':
        real_bc_r = real_data.reshape([len(real_locations), len(real_times)])[-1,:]

        real_bc_r[real_bc_r<0] = 0

    
    else:
        real_bc_r = None
    
    if args.u0_from_data:
        real_u0 = real_data.reshape([len(real_locations), len(real_times)])[:,0]

        real_u0[real_u0<0] = 0

    
    # locations, including added locations that can be used in synthetic 
    # case only
    if len(args.add_data_pts) > 0:
        locations = np.concatenate((real_locations, np.array(args.add_data_pts)))
        # reorder the locations
        locations = np.sort(locations)
        diff_locations = locations[:-1]
    else:
        locations = real_locations
    # times
    times = real_times
    # STEP 4: Create the PDE grid and coefficients grid
    #----------------------------------------------------
    # PDE and coefficients grids
    factor_L = 1.2 if args.rbc == 'zero' else 1.01
    L = locations[-1]*factor_L
    coarsening_factor = 5
    n_grid_c = 20
    grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)
    
    # Step 4.1: Create u0
    #-----------------------
    if args.u0_from_data:
        # interpolate real_u0 to the grid
        u0 = np.interp(grid, locations, real_u0)
    else:
        u0 = None
    
    # STEP 5: Create the PDE time steps array
    #------------------------------------------
    tau_max = 30*60 # Final time in sec
    cfl = 5 # The cfl condition to have a stable solution
             # the method is implicit, we can choose relatively large time steps 
    tau = create_time_steps(h, cfl, tau_max, args.adaptive)
    
    # STEP 6: Create the domain geometry
    #-------------------------------------
    G_c = create_domain_geometry(grid_c, args.inference_type)
    
    # STEP 7: Create the PDE form
    #----------------------------
    PDE_form = create_PDE_form(real_bc_l, real_bc_r,
                               grid, grid_c, grid_c_fine, n_grid, h, times,
                               args.inference_type,
                               u0=u0)
    # STEP 8: Create the CUQIpy PDE object
    #-------------------------------------
    PDE = TimeDependentLinearPDE(PDE_form,
                                 tau,
                                 grid_sol=grid,
                                 method='backward_euler', 
                                 grid_obs=locations,
                                 time_obs=times,
                                 data_grad=args.data_grad) 
    
    # STEP 9: Create the range geometry
    #----------------------------------
    if args.data_grad:
        G_cont2D = Continuous2D((diff_locations, times))
    else:
        G_cont2D = Continuous2D((locations, times))
    
    # STEP 10: Create the CUQIpy PDE model
    #-------------------------------------
    A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

    return A
def read_all_scenarios(scenario_dir, scenario_tag, physics):
    
    # Read all cases for the scenario
    animals = all_animals()
    ears = all_ears()
    data_diff_list = []
    data_adv_list = []
    for animal in animals:
        for ear in ears:
            # Read the experiment data
            data_diff_list.append(
                read_experiment_data(
                    scenario_dir+"/"+scenario_tag[0],
                    animal+"_"+ear+"_"+scenario_tag[2]
                    ))
            data_adv_list.append(
                read_experiment_data(
                    scenario_dir+"/"+scenario_tag[1],
                    animal+"_"+ear+"_"+scenario_tag[3]))
    return data_diff_list, data_adv_list

def plot_v3_fig1( data_diff_list, data_adv_list):
                
    # Create a figure with 10 rows and 3 columns. First column is for the
    # data, second column is for the prediction from the diffusion model,
    # and the third column is for the prediction from the advection model.
    fig, axs = plt.subplots(10, 3, figsize=(15, 30))
    #print(data_adv_list[0].keys())
    # dict_keys(['exact', 'exact_data', 'data', 'mean_recon_data', 'x_samples', 's_samples', 'experiment_par', 'locations', 'times', 'lapse_time', 'num_tree_node_list', 'epsilon_list'])
    for i in range(len(data_diff_list)):
        # Plot the data
        plt.sca(axs[i, 0])
        plot_time_series(data_diff_list[i]['times'], data_diff_list[i]['locations'],
                         data_diff_list[i]['data'])
        
        # Plot the prediction from the diffusion model
        plt.sca(axs[i, 1])
        plot_time_series(data_diff_list[i]['times'], data_diff_list[i]['locations'],
                         data_diff_list[i]['mean_recon_data'])
        
        # Plot the prediction from the advection model
        plt.sca(axs[i, 2])
        plot_time_series(data_adv_list[i]['times'], data_adv_list[i]['locations'],
                         data_adv_list[i]['mean_recon_data'])
        

def plot_v3_fig1_b( data_diff_list, data_adv_list):
                
    # Create a figure with 10 rows and 3 columns. First column is for the
    # data, second column is for the prediction from the diffusion model,
    # and the third column is for the prediction from the advection model.
    fig, axs = plt.subplots(10, 3, figsize=(15, 30))
    # print(data_adv_list[0].keys())
    # dict_keys(['exact', 'exact_data', 'data', 'mean_recon_data', 'x_samples', 's_samples', 'experiment_par', 'locations', 'times', 'lapse_time', 'num_tree_node_list', 'epsilon_list'])
    for i in range(len(data_diff_list)):
        # Plot the data
        plt.sca(axs[i, 0])
        real_times, real_locations, real_data, real_std_data, diff_locations, real_data_diff, real_std_data_diff = read_data_files(data_diff_list[i]['experiment_par'])
        plot_time_series(real_times, real_locations,
                         real_data.reshape(len(real_locations), len(real_times)))
        
        # Plot the prediction from the diffusion model
        #---
        A = create_A(data_diff_list[i]) 
        mean_recon_data = \
            A(data_diff_list[i]["x_samples"].funvals.mean(), is_par=False)
        non_grad_mean_recon_data = A.pde._solution_obs


        plt.sca(axs[i, 1])
        plot_time_series(real_times, real_locations,
                         non_grad_mean_recon_data)

        #---
        # Plot the prediction from the advection model
        #---
        try:
            A = create_A(data_adv_list[i])
            mean_recon_data = \
                A(data_adv_list[i]["x_samples"].funvals.mean(), is_par=False)
            non_grad_mean_recon_data = A.pde._solution_obs

            plt.sca(axs[i, 2])
            plot_time_series(real_times, real_locations,
                             non_grad_mean_recon_data)
        except:
            pass

        


    
# dir_name +'/output'+ tag+'/'+tag+'_var.pkl'
def plot_v3_fig2(data_diff_list, data_adv_list):
    # create a 10 by 3 plot, each row is for a different animal and ear
    # the first column is for the credibility interval of the inferred
    # diffusion parameter, the second column is for the prior and posterior
    # of the advection parameter, and the third column is for the prior and
    # posterior of the GIBBS parameter
    fig, axs = plt.subplots(10, 3, figsize=(15, 30))

    for i in range(len(data_diff_list)):
        # Plot the credibility interval of the inferred diffusion parameter
        plt.sca(axs[i, 0])
        data_diff_list[i]['x_samples'].funvals.plot_ci(68)
        
        try:
            cuqi.samples.Samples(data_adv_list[i]['x_samples'].samples[:-1,:], geometry=data_diff_list[i]['x_samples'].geometry).plot_ci( 68, plot_envelope_kwargs={'facecolor': 'g'}, color='g')
        except:
            pass
        
        # compute ESS min
        ESS_diff = np.min(data_diff_list[i]['x_samples'].compute_ess())
        try:
            ESS_adv = np.min(data_adv_list[i]['x_samples'].compute_ess())
        except:
            ESS_adv=0
            pass
        # add it to the plot
        plt.text(10, 1000, 'ESS: \n'+str(int(ESS_diff))+'\n'+str(int(ESS_adv)), fontsize=12)
        
        if ESS_adv != 0:
            # plot the prior and posterior of the advection parameter
            plt.sca(axs[i, 1])
            var_a_sqrt = 0.752**2
            var_a = 2*var_a_sqrt**2
            prior2 =cuqi.distribution.Gaussian(0, var_a) #TODO: store
            v_min = -3
            v_max = 3
            cuqi.utilities.plot_1D_density(prior2, v_min=v_min, v_max=v_max, color='b',label='prior')
            #plt.hist(data_adv_list[i]['x_samples'].samples[-1,:].flatten(), bins=50, alpha=0.5, label='$a$', color='orange')

            
            kde = sps.gaussian_kde(data_adv_list[i]['x_samples'].samples[-1,:].flatten())
            x = np.linspace(v_min, v_max, 100)
            l1 = plt.plot(x, kde(x), color='black', label='posterior')

            # plot the gibbs
            plt.sca(axs[i, 2])
            s = cuqi.distribution.Gamma(1.2, 5) #TODO: store
            s_samples = s.sample(10000)
            v_min = 0
            v_max = 10
            
            #cuqi.utilities.plot_1D_density(s, v_min=v_min, v_max=v_max, color='b',label='prior')
            kde_1 = sps.gaussian_kde(1/np.sqrt(s_samples.samples.flatten())) 
            kde_2 = sps.gaussian_kde(1/np.sqrt(data_adv_list[i]['s_samples'].samples.flatten()))
            x = np.linspace(v_min, v_max, 100)
            l1 = plt.plot(x, kde_1(x), color='blue', label='prior')
            l2 = plt.plot(x, kde_2(x), color='black', label='posterior')
            plt.legend()
        

        




In [ ]:
scenarios_dir = [
    "/Users/amal/Documents/research_code/CUQI-DTU/Collab-BrainEfflux-Data/A_Considered_for_the_paper/results_dec1",
    "/Users/amal/Documents/research_code/CUQI-DTU/Collab-BrainEfflux-Data/A_Considered_for_the_paper/results_nov30",
    "/Users/amal/Documents/research_code/CUQI-DTU/Collab-BrainEfflux-Data/A_Considered_for_the_paper/results_nov30"
    ]
scenario_tags_list = [
    ["paperV4CARealDiff_Gauess_gibbs_scale_all7",
     "paperV4CARealAdvDiff_Gauess_gibbs_scale_all7",
     "NUTSWithGibbs_constant_100.0_real_heterogeneous_2000_0.2_paperV4CARealDiff_Gauess_gibbs_scale_all7__0_5_0.1_fromDataClip",
     "NUTSWithGibbs_constant_100.0_real_advection_diffusion_2000_0.2_paperV4CARealAdvDiff_Gauess_gibbs_scale_all7__0_5_0.1_fromDataClip"],
    ["paperV4CARealDiff_Gauss4_gibbs_scale_all",
     "paperV4CARealAdvDiff_Gauss4_gibbs_scale_all",
     "NUTSWithGibbs_constant_100.0_real_heterogeneous_2000_0.2_paperV4CARealDiff_Gauss4_gibbs_scale_all__0_5_0.1_fromDataClip",
     "NUTSWithGibbs_constant_100.0_real_advection_diffusion_2000_0.2_paperV4CARealAdvDiff_Gauss4_gibbs_scale_all__0_5_0.1_fromDataClip"],
    ["paperV4CARealDiff_Gauss4_b_small_noise",
     None,
     "NUTS_constant_100.0_real_heterogeneous_2000_0.1_paperV4CARealDiff_Gauss4_b_small_noise__0_5_0.1_fromDataClip",
     None]


    ]

num_scenarios = len(scenarios_dir)
figs_to_plot = [[1,2]]*num_scenarios
physics = [['diffusion', 'advection-diffusion']]*num_scenarios

i = 1
data_diff_list, data_adv_list = read_all_scenarios(scenarios_dir[i], scenario_tags_list[i], physics[i])
if 1 in figs_to_plot[i]:
        plot_v3_fig1( data_diff_list, data_adv_list)
        # create directory if it does not exist
        import os
        os.makedirs('figures/'+scenario_tags_list[i][0], exist_ok=True)
        plt.savefig('figures/'+scenario_tags_list[i][0]+'/fig1.png')
        plot_v3_fig1_b( data_diff_list, data_adv_list)
        plt.savefig('figures/'+scenario_tags_list[i][0]+'/fig1_b.png')
        pass
if 2 in figs_to_plot[i]:
        plot_v3_fig2( data_diff_list, data_adv_list)
        plt.savefig('figures/'+scenario_tags_list[i][0]+'/fig2.png')

